<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/NLP_%D0%B2_%D0%97%D0%B4%D1%80%D0%B0%D0%B2%D0%BE%D0%BE%D1%85%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B8_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive

In [3]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

In [5]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np

In [6]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
to_find_df = pd.read_csv('drive/MyDrive/Применение NLP в Здравоохранении/data/to_find.tsv', sep='|')
concepts_df = pd.read_csv('drive/MyDrive/Применение NLP в Здравоохранении/data/концепты.tsv', sep='\t')

In [ ]:
to_find_df.head()

In [ ]:
concepts_df.head()

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Инициализация инструментов для предобработки текста
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    # Токенизация текста
    tokens = word_tokenize(text.lower())
    # Удаление стоп-слов и стемминг
    tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)

In [ ]:
# Применение предобработки к данным
to_find_df['processed_text'] = to_find_df['Text'].apply(preprocess_text)
concepts_df['processed_concept'] = concepts_df['CONCEPT'].apply(preprocess_text)

In [ ]:
to_find_df.head()

In [ ]:
concepts_df.head()

In [ ]:
# Векторизация текста
vectorizer = TfidfVectorizer()
text_vectors = vectorizer.fit_transform(to_find_df['processed_text'])
concept_vectors = vectorizer.transform(concepts_df['processed_concept'])

In [ ]:
# Вычисление косинусного сходства
similarity_matrix = cosine_similarity(text_vectors, concept_vectors)

In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]

In [ ]:
start_time = time.time()
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

elapsed_time = (time.time() - start_time) / 60

result.head(), elapsed_time

In [ ]:
result

Предобработка word2vec

In [ ]:
# Предобработка текста с помощью word2vec
def preprocess_for_word2vec(text):
    return simple_preprocess(text, deacc=True)

In [ ]:
# Применение предобработки
to_find_df['processed_tokens'] = to_find_df['Text'].apply(preprocess_for_word2vec)
concepts_df['processed_tokens'] = concepts_df['CONCEPT'].apply(preprocess_for_word2vec)

In [ ]:
# Объединение данных для обучения модели Word2Vec
all_texts = to_find_df['processed_tokens'].tolist() + concepts_df['processed_tokens'].tolist()

In [ ]:
# Обучение модели Word2Vec
word2vec_model = Word2Vec(sentences=all_texts, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Функция для получения среднего вектора текста
def get_average_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [ ]:
# Преобразование текстов в векторные представления
to_find_df['vector'] = to_find_df['processed_tokens'].apply(lambda tokens: get_average_vector(tokens, word2vec_model))
concepts_df['vector'] = concepts_df['processed_tokens'].apply(lambda tokens: get_average_vector(tokens, word2vec_model))

In [ ]:
# Вычисление косинусного сходства
similarity_matrix = cosine_similarity(np.vstack(to_find_df['vector']), np.vstack(concepts_df['vector']))

In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]

In [ ]:
# Создание таблицы с результатами
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

In [ ]:
result

Предобработка BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def get_bert_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
# Применение BERT для получения эмбеддингов текстов
to_find_df['bert_embeddings'] = to_find_df['Text'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))
concepts_df['bert_embeddings'] = concepts_df['CONCEPT'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))

In [ ]:
# Вычисление косинусного сходства между эмбеддингами
similarity_matrix = cosine_similarity(
    np.vstack(to_find_df['bert_embeddings']),
    np.vstack(concepts_df['bert_embeddings'])
)

In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]

In [ ]:
# Создание таблицы с результатами
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

In [ ]:
result

Вывод:

Использование BERT (Bidirectional Encoder Representations from Transformers) для оценки сходства между текстами существенно улучшает результаты Similarity благодаря его способности учитывать контекст и сложные семантические отношения.

In [ ]:
result.to_csv('result_BERT.csv', index=False, encoding='utf-8')